#### Task1
В качестве разрешенных выбраны следующие кодовые комбинации:<br>
  - *Е1=0000110111*
  - *Е2=0000101011*
  - *Е3=1000011001*
  - *Е4=0010010110*

Определить максимальную кратность обнаруживаемых и исправляемых ошибок.<br>
Показать процесс исправления ошибок максимальной кратности при передаче первой
разрешенной кодовой комбинации.

In [15]:
import numpy as np

def xor(a, b):
    return (a & (~b)) | ((~a) & b)

E1 = 0b0000110111
E2 = 0b0000101011
E3 = 0b1000011001
E4 = 0b0010010110

msg_list = np.array([0, E1, E2, E3, E4])

msg_list_len = len(msg_list)

d = np.array([[0] * msg_list_len] * msg_list_len)

d_list = []

for i in range(1, msg_list_len - 1):
    for j in range(i + 1, msg_list_len):
        d[i][j] = xor(msg_list[i], msg_list[j]).bit_count()
        print(f"d{i}{j} = {d[i][j]}")
        d_list.append(d[i][j])

d_min = min(d_list)

# t_ob <= d_min - 1
t_ob = d_min - 1
print(f"\nt_ob <= {t_ob}")

# t_isp <= (d_min - 1) / 2
t_isp = (d_min - 1) / 2
print(f"t_isp <= {t_isp}\n")


d12 = 3
d13 = 5
d14 = 3
d23 = 4
d24 = 6
d34 = 6

t_ob <= 2
t_isp <= 1.0

